In [ ]:
%pip install geopy

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy as gp


In [232]:
'''
(scrap)abnb_5-2020.csv
sep=','
id,listing_url,host_since,neighbourhood_cleansed,neighbourhood_group_cleansed,
latitude,longitude,property_type,room_type,bedrooms,bathrooms,guests_included,price,review_scores_rating

what we want:
id, neighbourhood_cleansed, neighbourhood_group_cleansed, latitude, longitude, property_type, room_type, bedrooms,
bathrooms

what we need to create:
last scraped or year
zipcodes


abnb_11-2021.csv
sep=','
id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,
host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,
host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,
host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,
neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,
accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,
minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,
minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,
availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,
number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,
review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,
review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,
calculated_host_listings_count,calculated_host_listings_count_entire_homes,
calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month


abnb_3_2022.gz --- these three have the same categories as above
abnb_6_2022.gz -- all the 2022 files are going to be in the same format
abnb_9_2022.csv ---this is what I worked through already

SyntaxError: invalid syntax (3805109947.py, line 1)

In [ ]:
'''
abnb_2015 through 2020.csv
sep = ','
id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,
price,minimum_nights,number_of_reviews,last_review,reviews_per_month,
calculated_host_listings_count,availability_365

what we want:
id, host_id, host_name, neighbourhood_group, neighbourhood, latitude, longitude, room_type,
calculated_host_listings_count, availability 365

what we need to create:
difference between neigbourhood or neighbourhood_group and the cleansed version
    cleansed is geocoded using lat and lon
last_scraped or some date column
    use csv file name? splice the string from the end using index
host_since?
    I think this is a nogo
host_listings_count vs calculated_host_listings_count
zipcodes

state, country columns can be created for entire dataset and assigned to Ny and US

'''

In [257]:
#df = pd.read_csv('inside_airbnb_zip.csv.gz', index_col = False, compression='gzip', low_memory = False)
copy_cols1 = ['id',
#            'scrape_id',
            'last_scraped',
            'host_id',
            'host_name',
#            'host_since',
#            'host_location',
            'calculated_host_listings_count',
#            'street',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
#            'state',
#            'zipcode',
#            'smart_location',
#            'country',
            'latitude',
            'longitude',
#            'property_type',
            'room_type',
#            'accommodates',
#            'bathrooms',
            'bedrooms',
            'availability_365']

copy_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood',
            'neighbourhood_group',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

rename_cols2 = ['id',
            'host_id',
            'host_name',
            'calculated_host_listings_count',
            'neighbourhood_cleansed',
            'neighbourhood_group_cleansed',
            'latitude',
            'longitude',
            'room_type',
            'availability_365']

read1 = ['data/abnb_9-2022.gz','data/abnb_6-2022.gz',
        'data/abnb_3-2022.gz','data/abnb_11-2021.csv',
         'data/abnb_2015.csv','data/abnb_2016.csv',
         'data/abnb_2017.csv','data/abnb_2018.csv',
         'data/abnb_2019.csv','data/abnb_2020.csv',]

#df = pd.read_csv(, skipinitialspace=True, usecols=copy_cols, low_memory = False)

counter = 0
for item in read1:
    counter += 1
    x=0
    x2=0
    y=0
    y2=0
    if item == read1[0]:
        df1 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        df1=df1.drop_duplicates(subset='id')
#        df = df.rename(columns = rename_cols)
    elif item in read1[1:4]: #change to 4 for 2021 csv
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
#        df2=df2.rename(columns = rename_cols2)
        df1 = pd.concat([df1, temp] , sort=False)
#        df1=df1.drop_duplicates(subset='id')
    elif item == read1[4]:
        df2 = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        df2=df2.drop_duplicates(subset='id')
        y_ins = item[10:14]
        df2['last_scraped'] = y_ins
    elif item in read1[5:10]:
        temp = pd.read_csv(item, skipinitialspace=True, usecols=copy_cols2, sep=',', low_memory = False)
        temp=temp.drop_duplicates(subset='id')
        y_ins = item[10:14]
        temp['last_scraped'] = y_ins
        df2 = pd.concat([df2, temp], sort=False)
    
#df2['zipcode'] = 'N/A'
df2['bedrooms'] = 'N/A'
df2 = df2.rename(columns={'neighbourhood_group':'neighbourhood_group_cleansed', 'neighbourhood': 'neighbourhood_cleansed'})
df2 = df2[['id', 'last_scraped', 'host_id', 'host_name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type',
       'bedrooms', 'availability_365', 'calculated_host_listings_count']]

data/abnb_2015.csv year is: 2015
data/abnb_2016.csv year is: 2016
data/abnb_2017.csv year is: 2017
data/abnb_2018.csv year is: 2018
data/abnb_2019.csv year is: 2019
data/abnb_2020.csv year is: 2020


,id,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,calculated_host_listings_count,availability_365,last_scraped
0,8843681,14104350,Irving,Brooklyn,Bedford-Stuyvesant,40.683633,-73.941417,Entire home/apt,3,353,2015
1,1571050,2637874,NaN,Brooklyn,Bedford-Stuyvesant,40.683758,-73.956439,Private room,2,338,2015
2,7272680,8792814,Caroline,Brooklyn,Bedford-Stuyvesant,40.692944,-73.946663,Private room,8,347,2015
3,1177725,6443617,Ramzy,Brooklyn,Bedford-Stuyvesant,40.679121,-73.952624,Entire home/apt,1,348,2015
4,5012587,266909,NaN,Brooklyn,Bedford-Stuyvesant,40.694618,-73.932694,Private room,1,301,2015
...,...,...,...,...,...,...,...,...,...,...,...
36918,46937371,27181542,Jt,Manhattan,Kips Bay,40.739610,-73.979710,Entire home/apt,15,180,2020
36919,46937954,287807769,Allen,Manhattan,Upper West Side,40.780610,-73.985140,Entire home/apt,1,83,2020
36920,46938514,303319010,Naiomy,Manhattan,East Harlem,40.800540,-73.933400,Shared room,1,176,2020
36921,46938537,3351317,Clara,Manhattan,Kips Bay,40.740330,-73.982420,Entire home/apt,3,83,2020


In [273]:
df1 = pd.concat([df1,df2], sort=False)

In [276]:
df1 = df1.rename(columns={'neighbourhood_group_cleansed': 'borough','neighbourhood_cleansed':'neighbourhood'})

In [278]:
df1['zipcode'] = 'N/A'

In [272]:
df2.keys()

Index(['id', 'last_scraped', 'host_id', 'host_name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'room_type',
       'bedrooms', 'availability_365', 'calculated_host_listings_count'],
      dtype='object')

In [ ]:
# Zipcode geolocator using the temp variable

def get_zipcode(df, geolocator, lat_field, lon_field, attempt=1, max_attempts=100):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]), timeout=None)
        return location.raw['address']['postcode']
    except KeyError:
        pass
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_zipcode (df, attempt=attempt+1)
        raise 
        
geolocator = gp.Nominatim(user_agent='sercavfer@gmail.com')

# Creating temp dataframe that holds all rows where zipcode is not formatted correctly or is incorrect
df1['zipcode'] = df1['zipcode'].fillna('N/A')
temp = df1[df1['zipcode'].str.len() != 5]

zipcodes = temp.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='longitude')

In [145]:
# Replacing zipcode values we filtered earlier with correct values from the geolocator

zip_copy = zipcodes.copy()
zip_copy = zip_copy.to_frame()
zip_copy = zip_copy.rename(columns = {0:'zipcode'})

zip_replace_index = df1.index.isin(zip_copy.index)

df.loc[zip_replace_index,'zipcode'] = zip_copy['zipcode']

In [146]:
# Using the NY_Zip_Codes.csv to filter our dataframe to only keep rows where zipcodes match one of the 5 boroughs

df['zipcode']=df['zipcode'].astype(int)

zips = pd.read_csv('data/NY_ZIP_Codes.csv')
zips = zips.drop(columns=['State FIPS','County Code','County FIPS', 'File Date'])

b_list= ['Bronx',
        'Richmond',
        'Queens',
        'Kings',
        'New York']

zips = zips[zips['County Name'].isin(b_list)]

df = df[df['zipcode'].isin(zips['ZIP Code'].unique())]

# To do
# Change borough values in df that don't match the zip-borough combination in zips

In [230]:
# Bulk of Feature Engineering should belong here
# assigns columns as date time..
# To do: Feature Engineer each column into appropriate formats

df['last_scraped'] = pd.to_datetime(df['last_scraped'], infer_datetime_format = True )
#df['host_since'] = pd.to_datetime(df['host_since'], infer_datetime_format = True )
df = df.rename(columns={'neighbourhood_group_cleansed': 'borough'})

# Using zips to assign borough values based on zipcode
#test_df = df.copy()
#test_zips = zips.copy()

zips.loc[(test_zips['County Name'] == 'Richmond'), 'County Name'] = 'Staten Island'
zips.loc[(test_zips['County Name'] == 'New York'), 'County Name'] = 'Manhattan'
zips.loc[(test_zips['County Name'] == 'Kings'), 'County Name'] = 'Brooklyn'

zips=zips.rename(columns={'ZIP Code':'zipcode'})
zips = zips.drop_duplicates(subset='zipcode')
df=df.merge(zips, on='zipcode', how='left')

df = df.drop(columns='borough')
df= df.rename(columns={'County Name':'borough'})

#df.dtypes

#df['zipcode'].sort_values(ascending=False)


id                                 int64
scrape_id                          int64
last_scraped              datetime64[ns]
host_id                            int64
host_name                         object
host_since                datetime64[ns]
host_location                     object
host_listings_count              float64
street                            object
neighbourhood_cleansed            object
state                             object
zipcode                            int64
smart_location                    object
country                           object
latitude                         float64
longitude                        float64
property_type                     object
room_type                         object
accommodates                       int64
bathrooms                        float64
bedrooms                         float64
availability_365                   int64
borough                           object
dtype: object

In [ ]:
from uszipcode import Zipcode
# Search zipcode within 30 miles, ordered from closest to farthest
result = search.by_coordinates(39.122229, -77.133578, radius=30, returns=5)
len(res) # by default 5 results returned

In [231]:
df['id'].unique()

array([   15509,     2515,     2539, ..., 29760980, 29185060, 29761111])

In [ ]:
# PANDAS DISPLAY OPTIONS

#pd.set_option('display.max_rows', None)
#pd.reset_option("all")

In [ ]:
# Bulk of Feature Engineering should belong here
# assigns columns as date time..
# To do: Feature Engineer each column into appropriate formats

df['last_scraped'] = pd.to_datetime(df['last_scraped'], infer_datetime_format = True )
df['host_since'] = pd.to_datetime(df['host_since'], infer_datetime_format = True )
df = df.rename(columns={'neighbourhood_group_cleansed': 'borough'})


#df.dtypes

#df['zipcode'].sort_values(ascending=False)


In [ ]:
# Replacing NaN's with N/A

df['host_listings_count'] = df['host_listings_count'].fillna('N/A')

In [ ]:
'''Random tests'''

#for item in copy_cols:
    #df[item].unique()
#df[df.duplicated(keep=False)] #checking which columns have duplicate values

# looking at quick overview of unique values in some of our categorical columns

unique_cols = ['host_location' , 
               'host_listings_count' , 
               'host_total_listings_count' , 
               'neighbourhood_cleansed' ,
               'smart_location' ,
               'borough' ,
               'property_type' , 
               'room_type' , 
               'accommodates' , 
               'bathrooms', 
               'country' ,
               'bedrooms']

for item in unique_cols:
    print(item , ' = ' , len(df[item].unique()) , 'unique values\n', df[item].unique(), '\n')
    
temp=df.groupby(['zipcode','borough'])['id'].count().to_frame().sort_values('id').head(50)
temp
######
#df[(df['zip_code'] != zips['']]

test_df['new'] = df['zipcode'].map(str) + df['borough'].map(str)
check_zips = dupl_zips.copy()
check_zips['new'] = check_zips['ZIP Code'].map(str) + check_zips['County Name'].map(str)

checking = test_df[~test_df['new'].isin(check_zips['new'].unique())]
checking

blah_zips = list(check_zips['new'])
blah_zips
#for item in blah_zips:
#    if (item in blah):
#        print(item)

# works!!!!!!!!!!!!!!!

try_list = list(test_df['zipcode'].unique())
dupl_zips = test_zips.drop_duplicates('ZIP Code')
loop_zips = dupl_zips.set_index('ZIP Code')['County Name'].to_frame()

#try_test = test_df[test_df['zipcode'].isin(try_list)]


for item in try_list:
    #test_df['BOROUGH'] = test_df[(test_df['zipcode'] == item), 'BOROUGH']
    repl = loop_zips.at[item,'County Name']
    test_df.loc[(test_df['zipcode'] == item), 'borough'] = repl
    
test_df['borough'].count()


##
keys = list(test_zips.columns.values)

i2 = test_df.set_index(keys).index

i1 = test_zips.set_index(keys).index

test_df['borough'] = test_df['borough'].map(test_zips.set_index('ZIP Code')['County Name'])
test_df.at[i,'HOUSING OCCUPANCY'] = borough[y]

#######

test_df = df.copy()

test_zips = zips.copy()



test_zips.loc[(test_zips['County Name'] == 'Richmond'), 'County Name'] = 'Staten Island'
test_zips.loc[(test_zips['County Name'] == 'New York'), 'County Name'] = 'Manhattan'
test_zips.loc[(test_zips['County Name'] == 'Kings'), 'County Name'] = 'Brooklyn'

test_zips=test_zips.rename(columns={'ZIP Code':'zipcode'})
test_zips = test_zips.drop_duplicates(subset='zipcode')
test_df=test_df.merge(test_zips, on='zipcode', how='left')


test_df = test_df.drop(columns='borough')
test_df= test_df.rename(columns={'County Name':'borough'})


In [ ]:
'''Original Geopy test ... probably irrelevant now'''

#def get_zipcode(df, geolocator, lat_field, lon_field):
#    location = geolocator.reverse((df[lat_field], df[lon_field]))
#    return location.raw['address']['postcode']


#geolocator = gp.Nominatim(user_agent='sercavfer@gmail.com')

#zipcodes = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='longitude')